# Train Models
<div style="color:red; font-size:14px;">!! Don't define functions here, import them from utils.py</div>

This notebook contains the code needed to train and store models to disk.

Remember that if you use a function with a random state you have to fix it to a number so that the results are reproducible.

## Imports

In [1]:
# Cython import
!python skseq/setup.py build_ext --build-lib=./skseq

running build_ext


In [2]:
import pandas as pd
import pickle

from skseq.id_feature import IDFeatures
from skseq.extended_feature import ExtendedFeatures

from skseq import structured_perceptron_c 
from skseq.structured_perceptron import StructuredPerceptron

from utils.utils import *

#from tensorflow.keras.callbacks import EarlyStopping
#from transformers import TFBertModel

In [20]:
%load_ext autoreload
%autoreload 2

## Create Train set

In [3]:
train = pd.read_csv("data/train_data_ner.csv")

In [4]:
X_train, y_train = get_data_target_sets(train)

Processing: 100%|██████████| 38366/38366 [00:38<00:00, 1002.58sentence/s]


### Create Corpus

We need to create our corpus using the training data. The corpus consists of two dictionaries, one for the words and one for the tags. The words dictionary maps each word to an index and the tags dictionary maps each tag to an index. We also need to create the reverse mapping for the tags dictionary. This is needed to convert the predictions back to the tag names.

Example:
```python
        sentences = [['I', 'love', 'Python'], ['Python', 'is', 'great']]
        tags = ['O', 'O', 'B']
        word_dict, tag_dict, tag_dict_rev = create_corpus(sentences, tags)
        # word_dict: {'I': 0, 'love': 1, 'Python': 2, 'is': 3, 'great': 4}
        # tag_dict: {'O': 0, 'B': 1}
        # tag_dict_rev: {0: 'O', 1: 'B'}
```

In [5]:
word_dict, tag_dict, tag_dict_rev = create_corpus(X_train, y_train)

### Create Training Sequence List

#### No Cython

In [6]:
train_seq = create_sequence_list(word_dict, tag_dict, X_train, y_train)

Adding sequences: 100%|██████████| 38366/38366 [05:30<00:00, 116.00sequence/s]


#### Cython

In [7]:
train_seq = create_sequence_listC(word_dict, tag_dict, X_train, y_train)

Adding sequences: 100%|██████████| 38366/38366 [05:31<00:00, 115.68sequence/s]


In [8]:
print(train_seq[0])
print(train_seq[0].to_words(sequence_list=train_seq))

0/0 1/0 2/0 3/0 4/0 5/0 6/1 7/0 8/0 9/0 10/0 11/0 12/1 13/0 14/0 9/0 15/0 1/0 16/2 17/0 18/0 19/0 20/0 21/0 
Thousands/O of/O demonstrators/O have/O marched/O through/O London/B-geo to/O protest/O the/O war/O in/O Iraq/B-geo and/O demand/O the/O withdrawal/O of/O British/B-gpe troops/O from/O that/O country/O ./O 


## Train Models

<div class="alert" style="padding: 20px;background-color: #2cbc84; color: white; margin-bottom: 15px;">
<h3>Structured Perceptron w/ Default Features</h3>
</div>

To train the structured perceptron we must create a feature mapper and build it.

In [9]:
feature_mapper = IDFeatures(train_seq)
feature_mapper.build_features()

In [10]:
show_features(feature_mapper, train_seq[0])

Initial features
[0] init_tag:O


Transition features
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[9] prev_tag:O::B-geo
[11] prev_tag:B-geo::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[9] prev_tag:O::B-geo
[11] prev_tag:B-geo::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[21] prev_tag:O::B-gpe
[23] prev_tag:B-gpe::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O
[3] prev_tag:O::O


Final features
[28] final_prev_tag:O


Emission features
[1] id:Thousands::O
[2] id:of::O
[4] id:demonstrators::O
[5] id:have::O
[6] id:marched::O
[7] id:through::O
[8] id:London::B-geo
[10] id:to::O
[12] id:protest::O
[13] id:the::O
[14] id:war::O
[15] id:in::O
[16] id:Iraq::B-geo
[17] id:and::O
[18] id:demand::O
[13] id:the::O
[19] id:withdrawal::O
[2] id:of::O
[20] id:British::B-gpe
[22] id:troops::O
[24] id:from::O
[25] id:that::O
[26] id:country::O
[27] id:.::O




### Train

#### No Cython

In [10]:
num_epochs = 15
sp = StructuredPerceptron(word_dict, tag_dict, feature_mapper)
sp.num_epochs = 5

In [11]:
%%time
sp.fit(feature_mapper.dataset, num_epochs)

Epoch: 0 Accuracy: 0.893815
Epoch: 1 Accuracy: 0.931674
Epoch: 2 Accuracy: 0.940913
Epoch: 3 Accuracy: 0.946175
Epoch: 4 Accuracy: 0.950018
Epoch: 5 Accuracy: 0.952577
Epoch: 6 Accuracy: 0.954425
Epoch: 7 Accuracy: 0.956033
Epoch: 8 Accuracy: 0.957185
Epoch: 9 Accuracy: 0.958481
Epoch: 10 Accuracy: 0.959217
Epoch: 11 Accuracy: 0.960524
Epoch: 12 Accuracy: 0.961121
Epoch: 13 Accuracy: 0.961207
Epoch: 14 Accuracy: 0.961983
CPU times: user 1h 33min 59s, sys: 20 s, total: 1h 34min 19s
Wall time: 1h 34min 18s


#### Cython

In [11]:
num_epochs = 15
sp_c = structured_perceptron_c.StructuredPerceptronC(word_dict, tag_dict, feature_mapper)
sp_c.num_epochs = 5

In [12]:
%%time
sp_c.fit(feature_mapper.dataset, num_epochs)

Epoch: 0 Accuracy: 0.893815
Epoch: 1 Accuracy: 0.931674
Epoch: 2 Accuracy: 0.940913
Epoch: 3 Accuracy: 0.946175
Epoch: 4 Accuracy: 0.950018
Epoch: 5 Accuracy: 0.952577
Epoch: 6 Accuracy: 0.954425
Epoch: 7 Accuracy: 0.956033
Epoch: 8 Accuracy: 0.957185
Epoch: 9 Accuracy: 0.958481
Epoch: 10 Accuracy: 0.959217
Epoch: 11 Accuracy: 0.960524
Epoch: 12 Accuracy: 0.961121
Epoch: 13 Accuracy: 0.961207
Epoch: 14 Accuracy: 0.961983
CPU times: user 59min 8s, sys: 13.5 s, total: 59min 21s
Wall time: 59min 20s


### Save

In [13]:
sp.save_model("fitted_models/01_SP_Default_Features")
sp_c.save_model("fitted_models/01C_SP_Default_Features")

<div class="alert" style="padding: 20px;background-color: #2cbc84; color: white; margin-bottom: 15px;">
<h3>Structured Perceptron w/ New Features</h3>
</div>

In [14]:
feature_mapper_ext = ExtendedFeatures(train_seq)
feature_mapper_ext.build_features()

In [15]:
show_features(feature_mapper_ext, train_seq[1])

Initial features
[0] init_tag:O


Transition features
[8] prev_tag:O::O
[45] prev_tag:O::B-tim
[47] prev_tag:B-tim::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[16] prev_tag:O::B-geo
[18] prev_tag:B-geo::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[59] prev_tag:O::B-org
[61] prev_tag:B-org::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[8] prev_tag:O::O
[16] prev_tag:O::B-geo
[77] prev_tag:B-geo::I-geo
[78] prev_tag:I-geo::O


Final features
[38] final_prev_tag:O


Emission features
[39, 2, 3] id:Helicopter::O
[39, 2, 3] firstupper::O
[39, 2, 3] alphanum::O
[40, 5, 3] id:gunships::O
[40, 5, 3] lower::O
[40, 5, 3] alphanum::O
[41, 42, 43, 44, 44] id:Saturday::B-tim
[41, 42, 43, 44, 44] firstupper::B-tim
[41, 42, 43, 44, 44] alphanum::B-tim
[41, 42, 43, 44, 44

### Train

#### No Cython

In [24]:
num_epochs = 15
sp_ext = StructuredPerceptron(word_dict, tag_dict, feature_mapper_ext)
sp_ext.num_epochs = 5

In [25]:
%%time
sp_ext.fit(feature_mapper_ext.dataset, num_epochs)

Epoch: 0 Accuracy: 0.930157
Epoch: 1 Accuracy: 0.944669
Epoch: 2 Accuracy: 0.948542
Epoch: 3 Accuracy: 0.951180
Epoch: 4 Accuracy: 0.953505
Epoch: 5 Accuracy: 0.954438
Epoch: 6 Accuracy: 0.955883
Epoch: 7 Accuracy: 0.956796
Epoch: 8 Accuracy: 0.957754
Epoch: 9 Accuracy: 0.958079
Epoch: 10 Accuracy: 0.959030
Epoch: 11 Accuracy: 0.959852
Epoch: 12 Accuracy: 0.959726
Epoch: 13 Accuracy: 0.960705
Epoch: 14 Accuracy: 0.961026
CPU times: user 1h 3min 28s, sys: 16.2 s, total: 1h 3min 44s
Wall time: 1h 3min 44s


#### Cython

In [16]:
num_epochs = 15
sp_ext_c = structured_perceptron_c.StructuredPerceptronC(word_dict, tag_dict, feature_mapper_ext)
sp_ext_c.num_epochs = 5

In [17]:
%%time
sp_ext_c.fit(feature_mapper_ext.dataset, num_epochs)

Epoch: 0 Accuracy: 0.930157
Epoch: 1 Accuracy: 0.944669
Epoch: 2 Accuracy: 0.948542
Epoch: 3 Accuracy: 0.951180
Epoch: 4 Accuracy: 0.953505
Epoch: 5 Accuracy: 0.954438
Epoch: 6 Accuracy: 0.955883
Epoch: 7 Accuracy: 0.956796
Epoch: 8 Accuracy: 0.957754
Epoch: 9 Accuracy: 0.958079
Epoch: 10 Accuracy: 0.959030
Epoch: 11 Accuracy: 0.959852
Epoch: 12 Accuracy: 0.959726
Epoch: 13 Accuracy: 0.960705
Epoch: 14 Accuracy: 0.961026
CPU times: user 1h 40s, sys: 12.1 s, total: 1h 52s
Wall time: 1h 53s


### Save

In [26]:
sp_ext.save_model("fitted_models/02_SP_Extended_Features")
sp_ext_c.save_model("fitted_models/02C_SP_Extended_Features")

<div class="alert" style="padding: 20px;background-color: #2cbc84; color: white; margin-bottom: 15px;">
<h3> BERT model. DL approach</h3>
</div>

### Upload train set

In [2]:
train_BERT = pd.read_csv("data/train_data_ner.csv")

### Grouping, Tokenizing and Padding


In [3]:
sentences, tag, enc_tag = process_BERT_data(train_BERT)
X_train = sentences
y_train = tag

In [4]:
input_ids,attention_mask = tokenize_BERT(X_train)

100%|██████████| 38366/38366 [00:08<00:00, 4402.22it/s]


### Testing Padding and Truncation Length

In [5]:
# TEST: Checking Padding and Truncation length's
was = list()
for i in range(len(input_ids)):
    was.append(len(input_ids[i]))
set(was)

{128}

In [6]:
# Train Padding
train_tag = list()
for i in range(len(y_train)):
    train_tag.append(np.array(y_train[i] + [0] * (128-len(y_train[i]))))
    
# TEST:  Checking Padding Length
was = list()
for i in range(len(train_tag)):
    was.append(len(train_tag[i]))
set(was)

{128}

### Building BERT Model: Transfer Learning

In [7]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
model = create_BERT_model(bert_model)

2023-06-03 09:53:47.322162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model che

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

### Train

In [9]:
early_stopping = EarlyStopping(mode='min',patience=5)
history_bert = model.fit([input_ids,attention_mask],
                         np.array(train_tag),
                         epochs = 25,batch_size = 30*2,
                         callbacks = early_stopping,verbose = True)

Epoch 1/25
  1/640 [..............................] - ETA: 7:40:52 - loss: 3.1836 - accuracy: 0.0382

KeyboardInterrupt: 

### Save

In [ ]:
model.save("fitted_models/bert_model.h5")